In [1]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [2]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as fct
from pyspark.sql import Row
from geopy.distance import distance as geo_dist
from pyspark.sql.types import DoubleType
import pandas as pd 

In [3]:
import getpass
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('final_proj-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '4g')
conf.set('spark.executor.instances', '15')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=yarn appName=final_proj-granoux>

In [4]:
spark = SparkSession(sc)

In [5]:
df = spark.read.csv('/datasets/project/istdaten/*/*/*', sep=';', header=True)

In [6]:
columns = 'TripDate string, TripId string, OperatorId string, OperatorAbbrv string, OperatorName string, ProductId string, LineId string, LineType string, UmlaufId string, TransportType string, AdditionalTrip boolean, FailedTrip boolean, BPUIC string, StopName string, ArrivalTimeScheduled string, ArrivalTimeActual string, ArrivalTimeActualStatus string,     DepartureTimeScheduled string, DepartureTimeActual string, DepartureTimeActualStatus string, SkipStation boolean'
columns = list(map(lambda x: x.split()[0],columns.split(',')))

In [7]:
for old, new in zip(df.columns, columns):
    print(old, new)
    df = df.withColumnRenamed(old, new)

BETRIEBSTAG TripDate
FAHRT_BEZEICHNER TripId
BETREIBER_ID OperatorId
BETREIBER_ABK OperatorAbbrv
BETREIBER_NAME OperatorName
PRODUKT_ID ProductId
LINIEN_ID LineId
LINIEN_TEXT LineType
UMLAUF_ID UmlaufId
VERKEHRSMITTEL_TEXT TransportType
ZUSATZFAHRT_TF AdditionalTrip
FAELLT_AUS_TF FailedTrip
BPUIC BPUIC
HALTESTELLEN_NAME StopName
ANKUNFTSZEIT ArrivalTimeScheduled
AN_PROGNOSE ArrivalTimeActual
AN_PROGNOSE_STATUS ArrivalTimeActualStatus
ABFAHRTSZEIT DepartureTimeScheduled
AB_PROGNOSE DepartureTimeActual
AB_PROGNOSE_STATUS DepartureTimeActualStatus
DURCHFAHRT_TF SkipStation


In [8]:
df.head()

Row(TripDate='13.09.2017', TripId='80:06____:17010:000', OperatorId='80:06____', OperatorAbbrv='DB', OperatorName='DB Regio AG', ProductId='Zug', LineId='17010', LineType='RE', UmlaufId=None, TransportType='RE', AdditionalTrip='false', FailedTrip='false', BPUIC='8500090', StopName='Basel Bad Bf', ArrivalTimeScheduled=None, ArrivalTimeActual=None, ArrivalTimeActualStatus='PROGNOSE', DepartureTimeScheduled='13.09.2017 05:45', DepartureTimeActual='13.09.2017 05:45:00', DepartureTimeActualStatus='PROGNOSE', SkipStation='false')

In [9]:
df.count()

196232995

In [10]:
df.select("StopName").distinct().count()

10366

### Metadata

We first start by read metadata in order to select stop station within 10 km from Zürich 

In [11]:
df_meta = spark.read.csv('/datasets/project/metadata')

In [12]:
#Here we can see that we have some duplicated stop name 
#df_meta.filter(df_meta._c0.contains('Zürich')).head(30)

In [15]:
df_meta.printSchema()

root
 |-- _c0: string (nullable = true)



In [16]:
df_meta = df_meta.select(fct.split(df_meta['_c0'], '  ')[1].alias('Long'), 
                         fct.split(fct.split(df_meta['_c0'], '  ')[2], ' ')[0].alias('Lat'), fct.split(df_meta['_c0'], '% ')[1].alias('StopName_Meta') )

In [17]:
df_meta.show()

+---------+---------+-------------------+
|     Long|      Lat|      StopName_Meta|
+---------+---------+-------------------+
|26.074412|44.446770|          Bucuresti|
| 1.811446|50.901549|             Calais|
| 1.075329|51.284212|         Canterbury|
|-3.543547|50.729172|             Exeter|
| 9.733756|46.922368|            Fideris|
| 8.571251|50.051219|Frankfurt Flughafen|
|18.643803|54.355520|             Gdansk|
| 7.389462|47.191804|           Grenchen|
|29.019602|40.996348|           Istanbul|
| 9.873959|48.577852|  Amstetten (Württ)|
| 4.786044|43.921937|            Avignon|
| 2.140369|41.378914|          Barcelona|
| 7.589551|47.547405|              Basel|
| 7.395229|46.937482|       Bern Bümpliz|
|-1.899480|52.483627|         Birmingham|
| 6.838953|46.949588|          Boudry TN|
|17.106466|48.158910|         Bratislava|
| 4.335694|50.835376|          Bruxelles|
|-2.979650|53.404289|          Liverpool|
| 8.500049|47.114619|         Lothenbach|
+---------+---------+-------------

In [18]:
#Again we can see that we have many occurance of Zurich with different coordinate
#df_meta.filter(df_meta.StopName_Meta == 'Zürich').show()

In [19]:
df_meta = df_meta.withColumn("Long", df_meta["Long"].cast(DoubleType()))
df_meta = df_meta.withColumn("Lat", df_meta["Lat"].cast(DoubleType()))

In [20]:
#Coordinate of the main station of Zürich
Lat_zu = 47.377941
Long_ZU = 8.540141
def dist_to_ZU(lat, long): 
    res = str(geo_dist((lat, long), (Lat_zu, Long_ZU)))
    res = round(float(res.split()[0]),1)
    return res

In [21]:
udf_dist_to_ZU = udf(dist_to_ZU, DoubleType()) 

In [22]:
df_meta = df_meta.withColumn("Dist in km", udf_dist_to_ZU('Lat','Long'))

In [23]:
df_meta.show()

+---------+---------+-------------------+----------+
|     Long|      Lat|      StopName_Meta|Dist in km|
+---------+---------+-------------------+----------+
|26.074412| 44.44677|          Bucuresti|    1395.6|
| 1.811446|50.901549|             Calais|     627.8|
| 1.075329|51.284212|         Canterbury|     694.5|
|-3.543547|50.729172|             Exeter|     957.2|
| 9.733756|46.922368|            Fideris|     103.7|
| 8.571251|50.051219|Frankfurt Flughafen|     297.3|
|18.643803| 54.35552|             Gdansk|    1050.8|
| 7.389462|47.191804|           Grenchen|      89.5|
|29.019602|40.996348|           Istanbul|    1777.5|
| 9.873959|48.577852|  Amstetten (Württ)|     166.5|
| 4.786044|43.921937|            Avignon|     482.8|
| 2.140369|41.378914|          Barcelona|     838.7|
| 7.589551|47.547405|              Basel|      74.1|
| 7.395229|46.937482|       Bern Bümpliz|      99.7|
| -1.89948|52.483627|         Birmingham|     938.9|
| 6.838953|46.949588|          Boudry TN|     

In [24]:
df_meta = df_meta.filter(df_meta['Dist in km'] < 10)
df_meta.show()

+--------+---------+--------------------+----------+
|    Long|      Lat|       StopName_Meta|Dist in km|
+--------+---------+--------------------+----------+
|8.521961|47.351679|Zimmerberg-Basist...|       3.2|
|8.434713|47.390882|              Urdorf|       8.1|
|8.437543|47.357432|      Birmensdorf ZH|       8.1|
|8.468175|47.325896| Bonstetten-Wettswil|       7.9|
| 8.43033|47.380971|   Urdorf Weihermatt|       8.3|
|8.463472|47.368305|             Waldegg|       5.9|
|8.513918|47.370293|              Zürich|       2.2|
|8.438705|47.338209|            Aesch ZH|       8.8|
|8.467781|47.315088|          Bonstetten|       8.9|
|8.437173|47.353936|      Birmensdorf ZH|       8.2|
|8.540192|47.378177|           Zürich HB|       0.0|
| 8.48894|47.391481|   Zürich Altstetten|       4.1|
|8.548466|47.366611|  Zürich Stadelhofen|       1.4|
|8.561372|47.350124|Zürich Tiefenbrunnen|       3.5|
|8.544115|47.411529|     Zürich Oerlikon|       3.7|
|8.544636|47.418747|      Zürich Seebach|     

In [25]:
StopName = df_meta.select(df_meta.StopName_Meta).distinct()

In [26]:
Df_Zurich = df.join(StopName, StopName.StopName_Meta == df.StopName, how='inner')

In [28]:
Df_Zurich.printSchema()

root
 |-- TripDate: string (nullable = true)
 |-- TripId: string (nullable = true)
 |-- OperatorId: string (nullable = true)
 |-- OperatorAbbrv: string (nullable = true)
 |-- OperatorName: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- LineId: string (nullable = true)
 |-- LineType: string (nullable = true)
 |-- UmlaufId: string (nullable = true)
 |-- TransportType: string (nullable = true)
 |-- AdditionalTrip: string (nullable = true)
 |-- FailedTrip: string (nullable = true)
 |-- BPUIC: string (nullable = true)
 |-- StopName: string (nullable = true)
 |-- ArrivalTimeScheduled: string (nullable = true)
 |-- ArrivalTimeActual: string (nullable = true)
 |-- ArrivalTimeActualStatus: string (nullable = true)
 |-- DepartureTimeScheduled: string (nullable = true)
 |-- DepartureTimeActual: string (nullable = true)
 |-- DepartureTimeActualStatus: string (nullable = true)
 |-- SkipStation: string (nullable = true)
 |-- StopName_Meta: string (nullable = true)



In [32]:
Df_Zurich = Df_Zurich.drop('StopName_Meta')

In [34]:
Df_Zurich.show()

+----------+--------------+----------+-------------+--------------------+---------+------+--------+--------+-------------+--------------+----------+-------+----------------+--------------------+-------------------+-----------------------+----------------------+-------------------+-------------------------+-----------+
|  TripDate|        TripId|OperatorId|OperatorAbbrv|        OperatorName|ProductId|LineId|LineType|UmlaufId|TransportType|AdditionalTrip|FailedTrip|  BPUIC|        StopName|ArrivalTimeScheduled|  ArrivalTimeActual|ArrivalTimeActualStatus|DepartureTimeScheduled|DepartureTimeActual|DepartureTimeActualStatus|SkipStation|
+----------+--------------+----------+-------------+--------------------+---------+------+--------+--------+-------------+--------------+----------+-------+----------------+--------------------+-------------------+-----------------------+----------------------+-------------------+-------------------------+-----------+
|13.09.2017|  85:11:10:002|     85:11|  